# Assignment 1: Travelling Salesman Problem
## (A) Core Implementation 

Implement a genetic algorithm with:

- Population initialisation
- Selection mechanism
- At least two different crossover operators
- At least two different mutation operators
- Fitness evaluation

In [3]:
import random
import numpy as np


In [4]:
# Constants
POPULATION_SIZE = 500
GENOME_SIZE = 52  # Number of cities
MUTATION_RATE = 0.05
CROSSOVER_RATE = 0.84
GENERATIONS = 100


In [14]:
def load_distance_matrix(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    # Skip header lines until NODE_COORD_SECTION
    start_index = lines.index('NODE_COORD_SECTION\n') + 1
    coordinates = []
    
    for line in lines[start_index:]:
        if line.strip() == 'EOF':
            break
        parts = line.strip().split()
        coordinates.append((float(parts[1]), float(parts[2])))
    
    # Calculate distance matrix
    num_cities = len(coordinates)
    matrix = np.zeros((num_cities, num_cities))
    
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                dist = np.sqrt((coordinates[i][0] - coordinates[j][0])**2 + (coordinates[i][1] - coordinates[j][1])**2)
                matrix[i][j] = dist
    
    return matrix

### Population initialisation

In [6]:
# Initialize population with random permutations of cities
def init_population(population_size, genome_size):
    population = []
    for _ in range(population_size):
        genome = list(range(genome_size))
        random.shuffle(genome)
        population.append(genome)
    return population


### Selection mechanism

In [7]:
# Fitness function: total distance of the tour
def fitness(genome, distance_matrix):
    total_distance = 0
    for i in range(len(genome)):
        total_distance += distance_matrix[genome[i-1]][genome[i]]
    return -total_distance  # Negative because we want to minimize distance

In [8]:

# Selection mechanism: roulette wheel selection
def select_parent(population, fitnesses):
    total_fitness = sum(fitnesses)
    pick = random.uniform(0, total_fitness)
    current = 0
    for individual, fitness in zip(population, fitnesses):
        current += fitness
        if current > pick:
            return individual


In [9]:
# Selection mechanism: roulette wheel selection
def select_parent(population, fitnesses):
    total_fitness = sum(fitnesses)
    pick = random.uniform(0, total_fitness)
    current = 0
    for individual, fitness in zip(population, fitnesses):
        current += fitness
        if current > pick:
            return individual

### Crossover

In [10]:

# Order Crossover (OX)
def crossover(parent1, parent2):
    if random.random() < CROSSOVER_RATE:
        start, end = sorted(random.sample(range(len(parent1)), 2))
        child1 = [None] * len(parent1)
        child1[start:end] = parent1[start:end]
        child2 = [None] * len(parent2)
        child2[start:end] = parent2[start:end]

        fill_child(child1, parent2, end)
        fill_child(child2, parent1, end)

        return child1, child2
    else:
        return parent1, parent2

def fill_child(child, parent, end):
    current_pos = end
    for gene in parent:
        if gene not in child:
            if current_pos >= len(child):
                current_pos = 0
            child[current_pos] = gene
            current_pos += 1

# Swap mutation
def mutate(genome):
    for i in range(len(genome)):
        if random.random() < MUTATION_RATE:
            j = random.randint(0, len(genome) - 1)
            genome[i], genome[j] = genome[j], genome[i]
    return genome


### Fitness evaluation 

In [11]:

# Genetic algorithm
def genetic_algorithm(distance_matrix):
    population = init_population(POPULATION_SIZE, GENOME_SIZE)

    for generation in range(GENERATIONS):
        fitness_values = [fitness(genome, distance_matrix) for genome in population]

        new_population = []
        for _ in range(POPULATION_SIZE // 2):
            parent1 = select_parent(population, fitness_values)
            parent2 = select_parent(population, fitness_values)
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.extend([mutate(offspring1), mutate(offspring2)])

        population = new_population

        fitness_values = [fitness(genome, distance_matrix) for genome in population]
        best_fitness = max(fitness_values)
        print(f"Generation {generation}: Best Fitness = {-best_fitness}")

    best_index = fitness_values.index(max(fitness_values))
    best_solution = population[best_index]
    print(f'Best Solution: {best_solution}')
    print(f'Best Fitness: {-fitness(best_solution, distance_matrix)}')


In [15]:
distance_matrix = load_distance_matrix('berlin52.txt')
genetic_algorithm(distance_matrix)


Generation 0: Best Fitness = 24703.504384172356


TypeError: object of type 'NoneType' has no len()